# cr3 CTF 2024
##   embedding | 376 pts

Task description:

```
This encoder seems strange, any idea how to break it?

author: Onirique, Duha
```

Attachments:

In [ ]:
import numpy as np
from numpy.polynomial import Polynomial

class Encoder:
  def __init__(self, deg: int):
    self.zeta = np.exp(2 * np.pi * 1j / deg)
    self.poly_degree = deg

  def pi(self, z: np.array) -> np.array:
    N = self.poly_degree // 16
    return z[:N]

  def encode(self, p: Polynomial) -> np.array:
    z = self.sigma(p)
    pi_z = self.pi(z)
    return pi_z

  def sigma(self, p: Polynomial) -> np.array:
    N = self.poly_degree // 2
    outputs = [p(self.zeta ** (2*i + 1)) for i in range(N)]
    return np.array(outputs)

flag = "cr3{" + "f" * 42 + "}"
assert flag[:4]=='cr3{' and flag[-1] == '}'

flag = flag[4:-1].encode()
pt = list(flag)
assert len(pt) == 42, len(pt)
p = Polynomial(pt)

encoder = Encoder(128)
ct = encoder.encode(p)
print(ct)
from math import floor
ct = [(floor(c.real * 10**8), floor(c.imag * 10**8)) for c in ct]
print(f"{ct = }")

# ct = array([1614.41597751+2440.04175266j, -239.31512831  +65.01569777j,
#        -174.3244422  +623.0315122j ,  148.33319848 +107.54945904j,
#         -80.39944861  -16.1430125j ,  106.05365816 +198.8020629j ,
#         252.91493127  +79.94326544j, -102.92505223 +220.19525344j])

In [14]:
ct = [
    (1614.41597751, 2440.04175266),
    (-239.31512831, 65.01569777),
    (-174.3244422, 623.0315122),
    (148.33319848, 107.54945904),
    (-80.39944861, -16.1430125),
    (106.05365816, 198.8020629),
    (252.91493127, 79.94326544),
    (-102.92505223, 220.19525344),
]

## Solution

We have a 42 degree polynomial and it's 8 evaluations on unit circle.

This is basic integer relation problem, but with several outputs(unlike the ordinary case where we have more precise single evaluation).

Also, since we have 8 complex points, thus we have 16 independant relations. I guess that's enough to LLL everything up.

let $\zeta = e^{\frac{2 * \pi * I}{128}}$, $rz_i = Re(\zeta^{2 * i + 1}), cz_i = Im(\zeta^{2 * i + 1})$

We have evaluations of polynomial $p$: $e_1 = p(\zeta), e_2 = p(\zeta^3), ..., e_8 = p(\zeta^15)$


We can construct a Lattice:

\begin{align*}
L &= \begin{pmatrix}
sc & & & & Re(\zeta^0) & Im(\zeta^0) & Re(\zeta^{3 * 0}) & Im(\zeta^{3 * 0}) &... & Re(\zeta^{15 * 0}) & Im(\zeta^{15 * 0}) \\
& sc & & & Re(\zeta^1) & Im(\zeta^1) & Re(\zeta^{3 * 1}) & Im(\zeta^{3 * 1}) &... & Re(\zeta^{15 * 1}) & Im(\zeta^{15 * 1}) \\
 & & \ddots &  & & & \vdots \\
&  & & sc & Re(\zeta^{41}) & Im(\zeta^{41}) & Re(\zeta^{3 * 41}) & Im(\zeta^{3 * 41}) & ... & Re(\zeta^{15 * 41}) & Im(\zeta^{15 * 41}) \\
-sc & -sc & \cdots & -sc & -Re(e_1) & -Im(e_1) & -Re(e_2) & -Im(e_2) & \cdots & -Re(e_8) & -Im(e_8)  \\
\end{pmatrix}, \\
\end{align*}


And we know that $\vec{p} = (f_0, f_1, f_2, ..., f_{41}, 1) \times L = (sc * f_0 - sc, sc * f_1 - sc, ..., sc * f_41 - sc, \alpha_1, \alpha_2, ..., \alpha_{16})$ is in $L$ and it's kind of small, depending on the sc value.

After playing with scale values for a while I chose it to be 100 so the errors $\alpha_i$ have the same magnitude.

Also I had to normalize the values up to precision, aka $10^8$

In [15]:
ct = [(round(x * 10**8), round(y * 10**8)) for x, y in ct] # normalize

In [16]:
zeta = e ** (2 * pi * I / 128)
zetas = [pow(zeta, 2 * i + 1) for i in range(8)] # get evalutation points

scale = 100
M = Matrix(42 + 1, 42 + 2 * len(ct))
M.set_block(0, 0, identity_matrix(42) * scale)
M.set_block(42, 0, Matrix([-scale] * 42))

In [17]:
for i in range(len(ct)):
    c = zetas[i]
    zetai = [
        (floor((c**i).real() * 10**8), floor((c**i).imag() * 10**8)) for i in range(42)
    ] # normalized powers of zeta_i

    M.set_block(0, 42 + 2 * i, Matrix([x[0] for x in zetai]).T)
    M.set_block(0, 43 + 2 * i, Matrix([x[1] for x in zetai]).T)
    M[42, 42 + 2 * i] = -ct[i][0]
    M[42, 43 + 2 * i] = -ct[i][1]

In [20]:
for l in M.LLL():
    #print(l)
    tmp = M.solve_left(l)
    if all(x > 0 for x in tmp) or all(x < 0 for x in tmp):
        print(tmp)

(76, 49, 108, 95, 49, 53, 95, 52, 108, 108, 95, 121, 48, 117, 95, 110, 51, 51, 100, 95, 116, 48, 95, 98, 114, 51, 52, 107, 95, 116, 104, 51, 95, 51, 110, 99, 48, 100, 51, 114, 33, 33, 1)


In [21]:
bytes([76, 49, 108, 95, 49, 53, 95, 52, 108, 108, 95, 121, 48, 117, 95, 110, 51, 51, 100, 95, 116, 48, 95, 98, 114, 51, 52, 107, 95, 116, 104, 51, 95, 51, 110, 99, 48, 100, 51, 114, 33, 33])

b'L1l_15_4ll_y0u_n33d_t0_br34k_th3_3nc0d3r!!'